In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
!pip install cornac
!pip install papermill
!pip install scrapbook
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [80]:
import sys
import os
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.models.cornac.cornac_utils import predict_ranking,predict
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]
Cornac version: 1.14.2


In [81]:
# top k items to recommend
TOP_K = 10

# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 100

#Load data

In [82]:
df_train = pd.read_csv('/content/drive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering/final_reviews_train_than30.csv')
df_test = pd.read_csv('/content/drive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering/final_reviews_test_than30.csv')
book_info = pd.read_csv('/content/drive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering/final_books_than30.csv')

In [83]:
df_train.info

<bound method DataFrame.info of               id   product_id     user_id         user_name  rating  \
0        6081499    3734909.0     88616.0  Trương Minh Đăng       5   
1       12731297   14961285.0  16848626.0     Phạm Văn Tùng       5   
2        7849857   68774130.0   8046945.0         Minh Giải       5   
3       15666018  152449148.0   1333575.0         Genny Lee       5   
4       17241576     655679.0  24504222.0         Nhật Việt       5   
...          ...          ...         ...               ...     ...   
140834   8532204   73687713.0  15354457.0        Phạm Quỳnh       5   
140835  16112531   32155084.0   7997368.0      Đăng Khoa Đỗ       5   
140836  15966492    4687527.0  17278568.0    Dương Nhật Lâm       4   
140837  14235142   86267527.0   9745026.0     Hà Tuấn Khanh       5   
140838  14465364   48895045.0    382863.0       Lê Ngọc Huy       5   

                                         content  thank_count  rating_average  \
0                                 

In [84]:
df_train = df_train[['user_id', 'product_id', 'rating']]

In [85]:
df_train.columns=['userID', 'itemID','rating']

In [86]:
df_train.head()

,userID,itemID,rating
0,88616.0,3734909.0,5
1,16848626.0,14961285.0,5
2,8046945.0,68774130.0,5
3,1333575.0,152449148.0,5
4,24504222.0,655679.0,5


In [87]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1423 non-null   int64  
 1   product_id      1423 non-null   float64
 2   user_id         1423 non-null   float64
 3   user_name       1417 non-null   object 
 4   rating          1423 non-null   int64  
 5   content         526 non-null    object 
 6   thank_count     1423 non-null   int64  
 7   rating_average  1423 non-null   float64
 8   count           1423 non-null   int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 100.2+ KB


In [88]:
df_test = df_test[['user_id', 'product_id', 'rating']]

In [89]:
df_test.columns=['userID', 'itemID','rating']

In [90]:
df_test.head()

,userID,itemID,rating
0,2238735.0,10075763.0,5
1,11992154.0,69764534.0,4
2,22405575.0,982037.0,5
3,16952419.0,75996783.0,5
4,18025738.0,874800.0,5


# Cornac Dataset

In [91]:
train_set = cornac.data.Dataset.from_uir(df_train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 2739
Number of items: 16526


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 6711 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


# Train the BPR model

In [92]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [93]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/100 [00:00<?, ?it/s]

Optimization finished!
Took 11.6140 seconds for training.


# Prediction and Evaluation -0 

In [ ]:
df_train.shape
df_test.shape
with Timer() as t:
  predictions = predict_ranking(bpr, df_test, usercol='userID', itemcol='itemID')
  predictions.head()
print("Took {} seconds for prediction.".format(t))
predictions1 = pd.merge(df_test, predictions, on=["userID", "itemID"], how= "inner")[["userID",	"itemID",	"prediction",	]]
#df_test1[df_test1['prediction'].isna()]
predictions1[predictions1['prediction'].isna()]
df_test1 = pd.merge(df_test ,predictions1, on=["userID", "itemID"], how= "left")[["userID",	"itemID",	"rating", "prediction"]]
df_test1 = df_test1[~df_test1['prediction'].isna()]
df_test2 = df_test1[["userID",	"itemID",	"rating"]]
predictions2 = df_test1[["userID",	"itemID",	"prediction"]]
predictions2[predictions2['prediction'].isna()]
eval_rmse = rmse(df_test2, predictions2)
eval_mae = mae(df_test2, predictions2)
eval_rsquared = rsquared(df_test2, predictions2)
eval_exp_var = exp_var(df_test2, predictions2)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')


(140839, 3)

In [ ]:
eval_rmse = rmse(df_test2, predictions2)
eval_mae = mae(df_test2, predictions2)
eval_rsquared = rsquared(df_test2, predictions2)
eval_exp_var = exp_var(df_test2, predictions2)
print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

RMSE:		3.366975
MAE:		3.101341
rsquared:	-51.290727
exp var:	-7.395394


In [ ]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, df_train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 65.1011 seconds for prediction.


#Prediction and Evaluation

In [ ]:
#all_predictions.to_csv('/content/drive/MyDrive/recomment_system/Do_an/all_predictions_BRP.csv', header=True, index=None)

In [ ]:
all_predictions.shape

(45179971, 3)

In [ ]:
all_predictions = pd.read_csv('/content/drive/MyDrive/recomment_system/Do_an/all_predictions_BRP.csv')

In [ ]:
k = 10
eval_map = map_at_k(df_test, all_predictions, col_prediction='prediction', k=k)
print("MAP:\t%f" % eval_map)

MAP:	0.007640


In [ ]:
k = 10
eval_ndcg = ndcg_at_k(df_test, all_predictions, col_prediction='prediction', k=k)
print("NDCG:\t%f" % eval_ndcg)

NDCG:	0.011700


In [ ]:
eval_precision = precision_at_k(df_test, all_predictions, col_prediction='prediction', k=k)
print("Precision@K:\t%f" % eval_precision)

Precision@K:	0.002966


In [ ]:
k=10
eval_recall = recall_at_k(df_test, all_predictions, col_prediction='prediction', k=k)
print("Recall@K:\t%f" % eval_recall)

Recall@K:	0.021316
